In [0]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from sparkmeasure import StageMetrics
from pyspark.sql import functions as F
from pyspark import SparkConf, SparkContext
from operator import add
import re

spark = SparkSession.builder.appName("task1").getOrCreate()

stagemetrics = StageMetrics(spark)

dbfs_fileStore_prefix = "/FileStore/tables"
prefix = "ontimeperformance"

In [0]:
# https://stackoverflow.com/questions/55453101/pyspark-error-analysisexception-cannot-resolve-column-name
def clean_column_names(df):
  tempList = [] #Edit01
  for col in df.columns:
      new_name = col.strip()
      new_name = "".join(new_name.split())
      new_name = new_name.replace('.','') 
      tempList.append(new_name) 

  return df.toDF(*tempList) 

In [0]:
def task_1_rdd(spark_session, flights_path, aircrafts_path):
  flights_rdd = spark.sparkContext.textFile(flights_path).map(lambda x: x.split(",")) 
  aircrafts_rdd = spark.sparkContext.textFile(aircrafts_path).map(lambda x: x.split(",")) 

  # remove header and any row with null
  header = aircrafts_rdd.first()
  aircrafts_rdd = aircrafts_rdd.filter(lambda x: len(x) == len(header))
  aircrafts_rdd = aircrafts_rdd.filter(lambda x: x != header)
  # remove entries which are not cessna
  aircrafts_rdd = aircrafts_rdd.filter(lambda x: x[2] == "CESSNA")
  # filter out cancelled flights
  flights_rdd = flights_rdd.filter(lambda x : x[5] != "")
  # get list of (tail_num, model) from aircraft
  aircrafts_rdd_small = aircrafts_rdd.map(lambda x: (x[0], x[4]))
  # turn aircraft_rdd_small's model into just 3 digits
  aircrafts_rdd_small = aircrafts_rdd_small.map(lambda x: (x[0], re.findall(r'\d{3}', x[1])[0]))
  # list of (tail_num, carrier_code) from flights
  flights_rdd_small = flights_rdd.map(lambda x: (x[5], x[0]))
  
  joined_rdd = aircrafts_rdd_small.join(flights_rdd_small)
  final = joined_rdd.map(lambda x: (x[1][0], 1))
  
  counts = final.reduceByKey(lambda x, y: x + y).takeOrdered(3, key=lambda x: -x[1])
  for i in range(len(counts)):
     print("Cessna {} \t {}".format((re.findall(r'\d{3}', counts[i][0])[0]), counts[i][1]))

In [0]:
stagemetrics.begin()

task_1_rdd(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_small.csv", f"{dbfs_fileStore_prefix}/{prefix}_aircrafts.csv")

stagemetrics.end()

Cessna 172 57
Cessna 210 48
Cessna 421 47

In [0]:
stagemetrics.begin()

task_1_rdd(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_medium.csv", f"{dbfs_fileStore_prefix}/{prefix}_aircrafts.csv")

stagemetrics.end()

Cessna 210 500
Cessna 172 444
Cessna 421 430

In [0]:
stagemetrics.begin()

task_1_rdd(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_large.csv", f"{dbfs_fileStore_prefix}/{prefix}_aircrafts.csv")

stagemetrics.end()

In [0]:
stagemetrics.begin()

task_1_rdd(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_massive.csv", f"{dbfs_fileStore_prefix}/{prefix}_aircrafts.csv")

stagemetrics.end()